In [4]:
import pandas as pd
import numpy as np
import time

### Transform raw microdata from Statistic Canada to usable format

Function to transform NTS microdata

In [3]:
def tranform_NTS(source,codebook):
    #insert source data and codebook
    code = pd.read_csv(codebook, header = 0)
    data = pd.read_csv(source,sep = ' ', header = None)
    #drop NULL value in code
    code = code.dropna(axis=0)
    #create dataframe with all columns
    columnList = []
    for index in code.index:
        column = code['Name'][index]
        columnList.append(column)
    final_df = pd.DataFrame(columns = columnList)
    for column in final_df:
    #run map with function for all column of the data
        indexStart = int((code['Start'][code['Name'] == column])-1)
        indexEnd = int(code['End'][code['Name'] == column])
        def transform_column(string):
            return string[indexStart:indexEnd]
        final_df[column] = data[0].map(transform_column, na_action=None)
    return final_df

Define path for all data file and codebook file

In [1]:
path_trip2021 = "\\final-travel-project\\Data\\2021-TXT\\trip.txt"
path_tripDecode2021 = "\\final-travel-project\\Data\\2021-TXT\\trip_decode.csv"

path_trip2020 = "\\final-travel-project\\Data\\2020-TXT\\trip.txt"
path_tripDecode2020 = "\\final-travel-project\\Data\\2020-TXT\\trip_decode.csv"

path_trip2019 = "\\final-travel-project\\Data\\2019-TXT\\trip.txt"
path_tripDecode2019 = "\\final-travel-project\\Data\\2019-TXT\\trip_decode.csv"

path_trip2018 = "\\final-travel-project\\Data\\2018-TXT\\trip.txt"
path_tripDecode2018 = "\\final-travel-project\\Data\\2018-TXT\\trip_decode.csv"

Create dataframe for transformed data for each year for each table in the dataset

In [5]:
trip_df_2021 = tranform_NTS(path_trip2021,path_tripDecode2021)
trip_df_2020 = tranform_NTS(path_trip2020,path_tripDecode2020)
trip_df_2019 = tranform_NTS(path_trip2019,path_tripDecode2019)
trip_df_2018 = tranform_NTS(path_trip2018,path_tripDecode2018)

FileNotFoundError: [Errno 2] No such file or directory: '\\final-travel-project\\Data\\2021-TXT\\trip_decode.csv'

Merge data of different year

In [36]:
trip_df = pd.concat([trip_df_2021, trip_df_2020, trip_df_2019, trip_df_2018], axis = 0,ignore_index = True)

### Transform data for trip  table

Convert invalid data to NULL

In [37]:
invalidData = {
    1:['6','7','8','9'],
    2:['96','97','98','99'],
    3:['996','997','998','999'],
    4:['9996','9997','9998','9999'],
    5:['99996','99997','99998','99999'],
    6:['999996','999997','999998','999999'],
    7:['9999996','9999997','9999998','9999999'],
    8:['99999996','99999997','99999998','99999999']
}
for column in trip_df:
    if (column == 'PersonTripWeight') or (column == 'TripWeight'): continue
    trip_df[column][trip_df[column].isin(invalidData[len(trip_df[column][0])])] = np.nan

Tranforms date datatype

In [38]:
todate = { '1': '/1/1', '2': '/4/1','3': '/7/1', '4': '/10/1' }
trip_df['TripYear'] = trip_df['TripYear'] + trip_df['Quarter'].map(todate)
trip_df = trip_df.rename(columns = {'TripYear':'Date'})

Drop unnecessary columns

In [39]:
trip_df = trip_df.drop(['MultiCountStay','TripID','EntryPort','TripWeight','PersonTripWeight','Quarter','DurationMainDes','EntryProv','CarrierOrig','ReturnTransMode','NumbCountStay', 'NumbStatesStay'], axis = 1)

Transform TripType

In [40]:
transform_TripType = {
    '01': "International sameday trip",
    '02': "International overnight trip",
    '03': "Domestic same day interprovincial trip",
    '04': "Domestic overnight interprovincial trip",
    '05': "Domestic same day intraprovincial trip",
    '06': "Domestic overnight intraprovincial trip"
}   
trip_df['TripType'] = trip_df['TripType'].map(transform_TripType)   

Transform OrigProv

In [41]:
transform_OrigProv = {
    '10':"Newfoundland and Labrador",
    '11': "Prince Edward Island",
    '12': "Nova Scotia",
    '13': "New Brunswick",
    '24': "Quebec",
    '35': "Ontario",
    '46': "Manitoba",
    '47': "Saskatchewan",
    '48': "Alberta",
    '59': "British Columbia",
    '60': "Yukon",
    '61': "Northwest Territories",
    '62': "Nunavut"
}
trip_df['OrigProv'] = trip_df['OrigProv'].map(transform_OrigProv)

Transform DesCount

In [42]:
transform_DestCoun = {'Greenland':'11304','Saint Pierre and Miquelon':'11666','United States':'11840','Belize': '12084','Costa Rica':'12188','El Salvador': '12222','Guatemala': '12320','Honduras': '12340','Mexico' :'12484','Nicaragua' :'12558','Panama' :'12591','Antigua and Barbuda': '13028','Bahamas' :'13044','Barbados' :'13052','Bermuda': '13060','Virgin Islands (British)': '13092','Cayman Islands' :'13136','Cuba':'13192','Dominica' :'13212','Dominican Republic' :'13214','Grenada': '13308' ,'Guadeloupe' :'13312','Haiti': '13332','Jamaica' :'13388' ,'Martinique': '13474' ,'Montserrat' :'13500' ,'Dutch West Indies, Netherlands Antilles, Saint Eustatius': '13518','Aruba, Bonaire, Curacao': '13533','Saint Maarten': '13534','West Indies NES': '13599','Puerto Rico': '13630','Saint Barthélemy': '13652','Saint Kitts and Nevis': '13659','Anguilla' :'13660','Saint Lucia' :'13662','Saint Martin': '13663','Saint Vincent and the Grenadines': '13670','Trinidad and Tobago': '13780','Turks and Caicos Islands': '13796','Virgin Islands (United States)': '13850','Argentina': '14032','Bolivia': '14068','Brazil': '14076','Chile': '14152','Colombia': '14170','Ecuador' :'14218','Falkland Islands (Malvinas)' :'14238','French Guiana': '14254','Guyana' :'14328','Paraguay': '14600','Peru': '14604','Suriname': '14740','Uruguay' :'14858','Venezuela': '14862','Austria': '21040','Belgium': '21056','France' :'21250','Germany': '21276','Liechtenstein': '21438','Luxembourg': '21442','Monaco': '21492','Netherlands': '21528','Switzerland': '21756','Bulgaria' :'22100','Belarus' :'22112' ,'Czech Republic': '22203','Estonia' :'22233','Hungary': '22348' ,'Latvia': '22428','Lithuania': '22440','Moldova' :'22498','Poland': '22616' ,'Romania' :'22642','Russian' :'Federation' '22643' ,'Slovakia' :'22703','Ukraine' :'22804','Denmark': '23208' ,'Faroe Islands': '23234' ,'Finland': '23246','Aland Islands' :'23248','Iceland' :'23352','Northern Ireland, Republic of Ireland' :'23372','Norway' :'23578','Sweden': '23752','United Kingdom, England, Scotland, Wales': '23826','Jersey' :'23832','Azores': '24007' ,'Albania': '24008','Canary Islands': '24013','Andorra' :'24020' ,'Madeira' :'24051','Bosnia and Herzegovina' :'24070','Croatia': '24191' ,'Gibraltar' :'24292','Greece': '24300','Holy See (Vatican City State)': '24336' ,'Italy': '24380','Malta' :'24470','Montenegro': '24499','Portugal':'24620','San Marino' :'24674','Yugoslavia' :'24688' ,'Slovenia' :'24705' ,'Spain': '24724' ,'Republic of Macedonia': '24807','Ascension Island': '31111','Tristan da Cunha': '31113','Chagos Archipelago': '31115' ,'Cape Verde' :'31132','Benin' :'31204' ,'Gambia': '31270','Ghana' :'31288' ,'Guinea' :'31324' ,'Côte d’Ivoire' :'31384' ,'Liberia': '31430' ,'Mali': '31466','Mauritania' :'31478','Niger': '31562' ,'Nigeria' :'31566','Guinea-Bissau': '31624','Saint Helena' :'31654' ,'Senegal': '31686','Sierra Leone' :'31694','Togo': '31768','Burkina Faso': '31854' ,'Burundi': '32108' ,'Comoros' :'32174','Ethiopia': '32231','Eritrea': '32232' ,'Djibouti' :'32262','Kenya': '32404','Madagascar': '32450','Malawi': '32454' ,'Mauritius' :'32480' ,'Mozambique': '32508','Reunion' :'32638','Rwanda': '32646' ,'Seychelles': '32690','Somalia': '32706' ,'Zimbabwe' :'32716','Uganda' :'32800' ,'Tanzania' :'32834' ,'Zambia' :'32894' ,'Algeria': '33012' ,'Libya': '33434' ,'Morocco': '33504' ,'Western Sahara': '33732' ,'Sudan': '33736','Tunisia' :'33788','Egypt' :'33818','Angola': '34024','Cameroon': '34120' ,'Central African Republic': '34140' ,'Chad': '34148','Republic of the Congo': '34178','The Democratic Republic of the Congo': '34180','Equatorial Guinea': '34226','Gabon' :'34266' ,'Sao Tome and Principe': '34678' ,'Botswana': '35072' ,'Lesotho': '35426','Namibia': '35516' ,'Republic of South Africa': '35710' ,'Swaziland' :'35748','Africa NES' :'36199' ,'Afghanistan' :'41004','Azerbaijan': '41031' ,'Bahrain': '41048','Armenia': '41051' ,'Cyprus': '41196' ,'Georgia, Country Palestine, West Bank, Gaza Strip': '41275' ,'Iran': '41364' ,'Iraq' :'41368' ,'Israel' :'41376','Kazakhstan': '41398' ,'Jordan': '41400' ,'Kuwait' :'41414' ,'Kyrgyzstan' :'41417' ,'Lebanon': '41422','Oman' :'41512' ,'Qatar': '41634' ,'Saudi Arabia': '41682' ,'Syria': '41760' ,'Tajikistan': '41762','United Arab Emirates' :'41784','Turkey' :'41792','Turkmenistan' :'41795','Uzbekistan': '41860' ,'Yemen' :'41887','China' :'42156' ,'Taiwan' :'42158','Tibet': '42273' ,'Hong Kong Special Administrative Region' :'42344' ,'Japan': '42392','North Korea' :'42408' ,'South Korea' :'42410' ,'Macao Special Administrative Region' :'42446','Mongolia' :'42496','Brunei Darussalam': '43096','Burma (Myanmar)': '43104','Cambodia': '43116','Indonesia' :'43360','Laos' :'43418' ,'Malaysia': '43458' ,'Philippines': '43608' ,'Singapore': '43702','Viet Nam' :'43704' ,'Thailand': '43764','Bangladesh' :'44050' ,'Bhutan' :'44064' ,'Sri Lanka' :'44144' ,'India': '44356','Maldives': '44462','Nepal': '44524','Pakistan' :'44586','American Samoa': '51016' ,'Australia': '51036','Solomon Islands' :'51090' ,'Christmas Island' :'51162' ,'Cook Islands': '51184' ,'Fiji' :'51242','French Polynesia': '51258' ,'Kiribati' :'51296' ,'Guam' :'51316' ,'Other Australian Islands': '51399' ,'Nauru' :'51520' ,'New Caledonia': '51540' ,'Vanuatu' :'51548' ,'New Zealand' :'51554','Niue': '51570','Northern Mariana Islands': '51580' ,'Federated States of Micronesia' :'51583' ,'Marshall Islands': '51584' ,'Palau': '51585' ,'Papua New Guinea': '51598','Pitcairn' :'51612','Bismark Archipelago': '51702' ,'Marquesas Island' :'51716' ,'Society Is., Tahiti' :'51730' ,'Tuamotu Islands': '51738' ,'Tubuai Islands' :'51740','Tonga' :'51776' ,'Tuvalu' :'51798','Oceania NES' :'51799' ,'Wallis and Futuna' :'51876','Samoa': '51882','Antarctica': '61010'}
transform_DestCoun_reserve = {val: key for (key, val) in transform_DestCoun.items()}
trip_df['DestCoun'] = trip_df['DestCoun'].map(transform_DestCoun_reserve)

In [43]:
transform_DestProv = {
    '10':"Newfoundland and Labrador",
    '11': "Prince Edward Island",
    '12': "Nova Scotia",
    '13': "New Brunswick",
    '24': "Quebec",
    '35': "Ontario",
    '46': "Manitoba",
    '47': "Saskatchewan",
    '48': "Alberta",
    '59': "British Columbia",
    '60': "Yukon",
    '61': "Northwest Territories",
    '62': "Nunavut"
}
trip_df['DestProv'] = trip_df['DestProv'].map(transform_DestProv)

In [44]:
transform_DestRegion = {
    '0001': 'Avalon',
    '0005': 'Eastern Newfoundland',
    '0010': 'Central Newfoundland',
    '0015': 'Western Newfoundland',
    '0020': 'Labrador',
    '0099': 'Not stated, Newfoundland and Labrador',
    '0101': 'Prince Edward Island',
    '0202': 'Yarmouth & Acadian Shores',
    '0206': 'South Shore',
    '0211': 'Halifax Metro',
    '0215': 'Eastern Shore',
    '0220': 'Cape Breton Island',
    '0225': 'Northumberland Shore',
    '0232': 'Fundy Shore & Annapolis Valley',
    '0299': 'Not stated, Nova Scotia',
    '0300': 'Appalachian Range',
    '0302': 'Miramichi River',
    '0304': 'Acadian Coastal Drive',
    '0308': 'Fundy',
    '0318': 'River Valley',
    '0399': 'Not stated, New Brunswick',
    '0401': 'Îles-de-la-Madeleine',
    '0405': 'Gaspésie',
    '0410': 'Bas-Saint-Laurent',
    '0415': 'Québec',
    '0420': 'Charlevoix',
    '0425': 'Chaudières-Appalaches',
    '0430': 'Centre-du-Québec',
    '0435': 'Mauricie',
    '0440': 'Cantons-de-l’Est',
    '0445': 'Montérégie',
    '0450': 'Lanaudière',
    '0455': 'Laurentides',
    '0460': 'Montréal',
    '0465': 'Outaouais',
    '0470': 'Abitibi-Témiscamingue',
    '0475': 'Saguenay-Lac-Saint-Jean',
    '0480': 'Manicouagan',
    '0485': 'Duplessis',
    '0491': 'James Bay',
    '0492': 'Nunavik',
    '0493': 'Eeyou Istchee',
    '0495': 'Laval',
    '0499': 'Not stated, Quebec',
    '0502': 'Southwest Ontario',
    '0506': 'Niagara Falls and Wine Country',
    '0511': 'Hamilton, Halton, Brant',
    '0516': 'Huron, Perth, Waterloo, Wellington',
    '0521': 'Greater Toronto Area',
    '0526': 'York, Durham, Hills of Headwaters',
    '0531': 'Bruce Peninsula, Southern Georgian Bay and Lake Simcoe',
    '0536': 'Kawartha and Northumberland',
    '0541': 'South Eastern Ontario',
    '0546': 'Ottawa and Countryside',
    '0551': 'Haliburton Highlands to the Ottawa Valley',
    '0556': 'Muskoka, Parry Sound and Algonquin Park',
    '0560': 'North East Ontario',
    '0565': 'North Central Ontario',
    '0570': 'North West Ontario',
    '0599': 'Not stated, Ontario',
    '0601': 'Winnipeg',
    '0605': 'South East Manitoba',
    '0610': 'South Central Manitoba',
    '0615': 'South West Manitoba',
    '0620': 'North Central Manitoba',
    '0625': 'Parklands',
    '0630': 'Interlake',
    '0635': 'Northern Manitoba',
    '0699': 'Not stated, Manitoba',
    '0701': 'Southwestern Saskatchewan',
    '0705': 'Southeastern Saskatchewan',
    '0710': 'Regina',
    '0715': 'West Central Saskatchewan',
    '0720': 'Saskatoon',
    '0725': 'East Central Saskatchewan',
    '0730': 'Northern Saskatchewan',
    '0799': 'Not stated, Saskatchewan',
    '0801': 'Alberta North',
    '0805': 'Alberta Central',
    '0810': 'Edmonton and Area',
    '0815': 'Canadian Rockies',
    '0820': 'Calgary and Area',
    '0825': 'Alberta South',
    '0899': 'Not stated, Alberta',
    '0901': 'Vancouver Islands',
    '0905': 'Vancouver, Coast & Mountains',
    '0910': 'Thompson-Okanagan',
    '0915': 'Kootenay Rockies',
    '0920': 'Cariboo Chilcotin Coast',
    '0925': 'Northern B.C.',
    '0981': 'Whitehorse',
    '0982': 'Dawson',
    '0983': 'Rural Yukon',
    '0991': 'Northwest Territories',
    '0992': 'Nunavut'
}
trip_df['DestRegion'] = trip_df['DestRegion'].map(transform_DestRegion)

In [45]:
transform_DestCMA = {
    '0000': 'All Other Areas',
    '0001': 'St. John’s',
    '0205': 'Halifax',
    '0305': 'Moncton',
    '0310': 'Saint John',
    '0408': 'Saguenay',
    '0421': 'Quebec',
    '0433': 'Sherbrooke',
    '0442': 'Trois-Rivières',
    '0462': 'Montreal',
    '0505': 'Ottawa-Gatineau',
    '0521': 'Kingston',
    '0529': 'Peterborough',
    '0532': 'Oshawa',
    '0535': 'Toronto',
    '0537': 'Hamilton',
    '0539': 'St. Catharines - Niagara',
    '0541': 'Kitchener',
    '0543': 'Brantford',
    '0550': 'Guelph',
    '0555': 'London',
    '0559': 'Windsor',
    '0568': 'Barrie',
    '0580': 'Greater Sudbury',
    '0595': 'Thunder Bay',
    '0602': 'Winnipeg',
    '0705': 'Regina',
    '0725': 'Saskatoon',
    '0825': 'Calgary',
    '0835': 'Edmonton',
    '0915': 'Kelowna',
    '0932': 'Abbotsford',
    '0933': 'Vancouver',
    '0935': 'Victoria'
}
trip_df['DestCMA'] = trip_df['DestCMA'].map(transform_DestCMA)

In [46]:
transform_Reason = {
    '01': 'Holiday, leisure or recreation',
    '02': 'Visit friends or relatives',
    '03': 'Non-business conference, convention or trade show',
    '04': 'Shopping, non-routine',
    '05': 'To go to school or study, non-routine',
    '06': 'Medical, non-routine',
    '07': 'Religious reasons, non-routine',
    '08': 'Other personal, non-routine',
    '09': 'Business meeting, conference, convention, trade show',
    '10': 'Other business, non-routine'
}
trip_df['Reason'] = trip_df['Reason'].map(transform_Reason)

In [47]:
transform_AgeGroup = {
    '01': '18 - 24 years old',
    '02': '25 - 34 years old',
    '03': '35 - 44 years old',
    '04': '45 - 54 years old',
    '05': '55 - 64 years old'
}
trip_df['AgeGroup'] = trip_df['AgeGroup'].map(transform_AgeGroup)

In [48]:
transform_Gender = {
    '1': 'Male',
    '2': 'Female'
}
trip_df['Gender'] = trip_df['Gender'].map(transform_Gender)

In [49]:
transform_IncomeGroup = {
    '1': 'Less than $50,000',
    '2': '$50,000 to less than $70,000',
    '3': '$70,000 to less than $100,000',
    '4': '$100,000 and over'
}
trip_df['IncomeGroup'] = trip_df['IncomeGroup'].map(transform_IncomeGroup)

In [50]:
transform_MainTrans = {
    '01': 'Commercial airline',
    '02': 'Car',
    '03': 'Bus',
    '04': 'Train',
    '05': 'Commercial boat or cruise ship',
    '06': 'Ferry',
    '07': 'Other'
}
trip_df['MainTrans'] = trip_df['MainTrans'].map(transform_MainTrans)

Transform datatype for all numeric columns

In [51]:
#transform continuous data to float, invalid data is set to Null
numericColumn = ['Distance','PackageCostDom','PackageCostInter','SpendComTransCan','SpendLocTransCan', 'SpendRentalVehCan', 'SpendGasRepairCan','SpendAccomCan', 'SpendFBResCan', 'SpendFBStoCan', 'SpendRecCan','SpendEntCan', 'SpendAppeCan', 'SpendingOtherCan', 'FareTransIOCan','FareTransBetCountries', 'SpendComTrans', 'SpendRentalVeh','SpendGasRepair', 'SpendAccom', 'SpendFBRes', 'SpendFBSto', 'SpendRec','SpendEnt', 'SpendAppe', 'SpendingOther','TotalSpendCan','TotalSpendOut']
trip_df[numericColumn] = trip_df[numericColumn].astype(float)

In [52]:
#transform some columns to interger, invalid data is set to Null
intColumn = ['Duration','AdultOnTrip','ChildOnTrip','NumbCityStayCan']
trip_df[intColumn] = trip_df[intColumn].astype('Int64')

In [53]:
#transfer column to Boolean
columnList = ['NightOutsideCan','TravelPackage', 'ActVisitF', 'ActShopping', 'ActSightseeing','ActMuseum', 'ActHisSite', 'ActZoo', 'ActSport', 'ActFestival','ActPerformance', 'ActCasino', 'ActThemePark', 'ActAboEvent','ActMovie', 'ActDineout', 'ActWildlifeView', 'ActNationalPark','ActMedical', 'ActBusiness', 'ActAllTerrainVehical', 'ActBoating','ActCanoeing', 'ActCamping', 'ActHiking', 'ActFishing', 'ActBeach','ActHunting', 'ActGolfing', 'ActCycling', 'ActSnowmobiling','ActSkiingDownhill', 'ActSkiingCrossCountry', 'ActPlaySport','ActOther', 'ActNone']
transform_boolean = {
    '1': True,
    '2': False,
}
for column in columnList:
    trip_df[column] = trip_df[column].map(transform_boolean)

### Export transformed data to cvs file

In [54]:
trip_df.to_csv("trip_NTS.csv",index=False)